In [1]:
from pathlib import Path

from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI


FILE_FOLDER = Path('/home/peugomes/projetoWSLnote/ChatPDF/teste.ipynb').parent / 'files'
MODEL_NAME = 'gpt-3.5-turbo-0125'


def import_documents():
    documents = []
    for file in FILE_FOLDER.glob('*.pdf'):
        loader = PyPDFLoader(str(file))
        file_documents = loader.load()
        documents.extend(file_documents)
        return documents
    
    
def split_documents(documents):
    recur_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2500,
        chunk_overlap=250,
        separators=["\n\n", "\n", " ", ""]
    )
    
    documents = recur_splitter.split_documents(documents)
    
    for i, doc in enumerate(documents):
        doc.metadata['source'] = doc.metadata['source'].split('/')[-1]
        doc.metadata['doc_id'] = i
    return documents


def create_vector_store(documents):
    embedding_model = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(
        documents=documents,
        embedding=embedding_model
    )
    return vector_store




def create_chat_chain(vector_store):
    chat = ChatOpenAI(model=MODEL_NAME)
    memory = ConversationBufferMemory(return_messages=True)
    
    retriever = vector_store.as_retriever()
    chat_chain = ConversationalRetrievalChain.from_llm(
        llm=chat,
        memory=memory,
        retriever=retriever,
        return_source_documents=True,
        verbose=True)
    
    return chat_chain


In [8]:
documents = import_documents()
documents = split_documents(documents)
vector_store = create_vector_store(documents)
chain = create_chat_chain(vector_store)

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)